# 4. Distance Measures

In [1]:
#데이터 가공 도구
import numpy as np
import pandas as pd
import random
from datetime import datetime
random.seed(314)
np.random.seed(314)

import re

from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels

#시각화 도구
from IPython.core.pylabtools import figsize #ipython 사용시 그래프 출력크기 설정
from matplotlib import pyplot as plt

#matplotlib 기본 설정
import matplotlib
import matplotlib.font_manager 
%matplotlib inline
matplotlib.rc('font', family='NanumGothic', size = 12)

#x축, y축 그리드
matplotlib.rcParams['grid.color'] = 'gray'#그리드 색상
matplotlib.rcParams['grid.linestyle'] = 'dotted'#그리드 선 유형
matplotlib.rcParams['grid.linewidth'] = 0.5#그리드 선 굵기
#좌표 값의 설정
matplotlib.rcParams['axes.facecolor'] = '#F0F0F0'#도표 내 색상
matplotlib.rcParams['axes.axisbelow'] = False #그리드를 차트에서 항상위로 할 경우 False 로 설정
matplotlib.rcParams['axes.labelcolor'] = 'k'#x,y 축의 글 색상
matplotlib.rcParams['axes.grid'] = True#그리드 사용여부
matplotlib.rcParams['axes.edgecolor'] = '#E6E6E6'#도표 윤곽선 색상
matplotlib.rcParams['axes.unicode_minus'] = False#도표값에 -를 unicode를 사용할지 여부, 깨지면 False 추천
matplotlib.rcParams['xtick.color'] = 'gray'#x축 라벨 색상
matplotlib.rcParams['ytick.color'] = 'gray'#y축 라벨 색상
matplotlib.rcParams['xtick.alignment'] ='center' #x축 라벨 위치 정렬

In [2]:
retail = pd.read_excel('../../data/online_retail_II.xlsx')

In [3]:
print(retail.shape)
display(retail.head())

(525461, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
retail.isna().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [5]:
retail_raw = retail[~retail['Customer ID'].isna()]

In [6]:
retail_raw.loc[:, 'Customer ID'] = retail_raw['Customer ID'].astype(int)
retail_raw.loc[:, 'Customer ID'] = retail_raw['Customer ID'].astype(str)

/root/workspace/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [7]:
def remove_digit(x):
    return re.sub('[0-9]', '', x)

def remove_char(x):
    return re.sub('[a-bA-Z]','', x)

In [8]:
retail_raw.loc[:, 'Cancel'] = retail_raw['Invoice'].apply(lambda x: remove_digit(str(x)))
retail_raw.loc[:, 'Invoice'] = retail_raw['Invoice'].apply(lambda x: remove_char(str(x)))
retail_raw.loc[:, 'StockOption'] = retail_raw['StockCode'].apply(lambda x: remove_digit(str(x)))
retail_raw.loc[:, 'StockCode'] = retail_raw['StockCode'].apply(lambda x: remove_char(str(x)))

/root/workspace/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [9]:
retail_raw.loc[:, 'Country'] = retail_raw['Country'].astype('category')
retail_raw.loc[:, 'CountryLabel'] = retail_raw['Country'].cat.codes

In [10]:
retail_raw['StockOption'].unique()

array(['', 'P', 'W', 'C', 'B', 'F', 'L', 'S', 'A', 'N', 'POST', 'E', 'J',
       'D', 'G', 'LP', 'BL', 'K', 'H', 'GR', 'M', 'U', 'R', 'V', 'T', 'I',
       'BANK CHARGES', 'O', 'Z', 'TEST', 'PADS', 'ADJUST', 'SP', 'Y'],
      dtype=object)

In [11]:
remove_option = ['POST','BANK CHARGES','TEST', 'PADS', 'ADJUST']
for i in remove_option:
    set_remove = retail_raw['StockOption'] == i
    retail_raw = retail_raw[~set_remove]

In [12]:
set_remove = retail_raw['StockCode'] == ''
retail_raw = retail_raw[~set_remove]

In [21]:
retail_raw

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Cancel,StockOption,CountryLabel
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,,,34
1,489434,79323,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,,P,34
2,489434,79323,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,,W,34
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,,,34
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,,,34
...,...,...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530,United Kingdom,,,34
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530,United Kingdom,,,34
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530,United Kingdom,,,34
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530,United Kingdom,,,34


In [28]:
group_ = retail_raw.groupby(['Customer ID', 'StockCode', 'Invoice', 'InvoiceDate'])
retail_stock= group_.agg({'Quantity': sum}).reset_index()

In [29]:
set_outquant = retail_stock['Quantity'] > retail_stock['Quantity'].quantile(0.99)
set_minusquant = retail_stock['Quantity'] < 0
retail_stock = retail_stock[~set_outquant]
retail_stock = retail_stock[~set_minusquant]

/root/workspace/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [30]:
retail_stock.columns

Index(['Customer ID', 'StockCode', 'Invoice', 'InvoiceDate', 'Quantity'], dtype='object')

In [31]:
retail_stock[['Customer ID', 'StockCode', 'Invoice', 'InvoiceDate']].describe()

,Customer ID,StockCode,Invoice,InvoiceDate
count,377346,377346,377346,377346
unique,4256,3259,18596,17479
top,14911,85123,526089,2010-10-08 12:14:00
freq,5387,2985,229,229
first,NaN,NaN,NaN,2009-12-01 07:45:00
last,NaN,NaN,NaN,2010-12-09 20:01:00


In [32]:
retail_stock['Quantity'].describe().round()

count    377346.0
mean         10.0
std          16.0
min           1.0
25%           2.0
50%           6.0
75%          12.0
max         144.0
Name: Quantity, dtype: float64

In [33]:
def nunique(x):
    return len(np.unique(x))

def nunique_date(x):
    x = x.apply(lambda x: datetime.date(x))
    return len(np.unique(x))

retail_stock['popular'] = retail_stock.groupby('StockCode')['Customer ID'].transform(nunique)
retail_stock['products'] = retail_stock.groupby('Customer ID')['StockCode'].transform(nunique)
retail_stock['transactions'] = retail_stock.groupby(['Customer ID', 'StockCode'])['Invoice'].transform(nunique)
retail_stock['dates'] = retail_stock.groupby(['Customer ID', 'StockCode'])['InvoiceDate'].transform(nunique_date)

In [34]:
set_popular = (retail_stock['popular'] > 99)
set_products = (retail_stock['products'] > 9)
retail_popular = retail_stock[set_popular & set_products]

In [35]:
retail_popular[['Customer ID', 'StockCode', 'Invoice', 'InvoiceDate']].describe()

,Customer ID,StockCode,Invoice,InvoiceDate
count,282279,282279,282279,282279
unique,3680,908,17233,16298
top,14911,85123,525928,2010-10-07 16:48:00
freq,3959,2932,161,161
first,NaN,NaN,NaN,2009-12-01 07:45:00
last,NaN,NaN,NaN,2010-12-09 20:01:00


In [36]:
retail_popular.describe().round()

,Quantity,popular,products,transactions,dates
count,282279.0,282279.0,282279.0,282279.0,282279.0
mean,11.0,282.0,209.0,3.0,3.0
std,16.0,171.0,272.0,6.0,3.0
min,1.0,100.0,10.0,1.0,1.0
25%,2.0,162.0,68.0,1.0,1.0
50%,6.0,232.0,128.0,2.0,2.0
75%,12.0,350.0,228.0,3.0,3.0
max,144.0,1137.0,1612.0,112.0,38.0


In [37]:
retail_popular# 추천데이터, user * 상품. 구매일수.

,Customer ID,StockCode,Invoice,InvoiceDate,Quantity,popular,products,transactions,dates
0,12346,15056,499763,2010-03-02 13:08:00,3,350,22,1,1
1,12346,20679,499763,2010-03-02 13:08:00,1,187,22,1,1
2,12346,20682,499763,2010-03-02 13:08:00,1,210,22,1,1
3,12346,20685,513774,2010-06-28 13:53:00,1,472,22,1,1
4,12346,21523,513774,2010-06-28 13:53:00,1,234,22,1,1
...,...,...,...,...,...,...,...,...,...
389677,18287,84879,508581,2010-05-17 11:55:00,8,612,74,1,1
389679,18287,85040,508581,2010-05-17 11:55:00,24,162,74,2,2
389680,18287,85040,534346,2010-11-22 11:51:00,48,162,74,2,2
389682,18287,85199,508581,2010-05-17 11:55:00,24,170,74,1,1


In [38]:
def cat_codes(x):
    return x.astype('category').cat.codes
retail_popular.loc[:, 'StockLabel'] = retail_popular['StockCode'].transform(cat_codes)

/root/workspace/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/root/workspace/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [39]:
from sklearn.model_selection import train_test_split
cust_train, cust_test = train_test_split(retail_popular['Customer ID'].unique()
                                    , test_size = 0.33
                                    , random_state = 314
                                    , shuffle = True)

train_dict = {'cust_id': cust_train, 'train_yn': 1}
test_dict = {'cust_id': cust_test, 'train_yn': 0}
user_split = pd.concat((pd.DataFrame(train_dict), pd.DataFrame(test_dict)), axis=0)

retail_popular = retail_popular.merge(user_split.set_index('cust_id')
                     , left_on='Customer ID'
                     , right_index=True, how='left')

set_train = retail_popular['train_yn'] == 1
set_test = retail_popular['train_yn'] == 0
retail_train = retail_popular[set_train]
retail_test = retail_popular[set_test]

retail_train.loc[:, 'CustomerLabel'] = retail_train['Customer ID'].transform(cat_codes)
retail_test.loc[:, 'CustomerLabel'] = retail_test['Customer ID'].transform(cat_codes)

retail_popular = pd.concat((retail_train, retail_test), axis=0)

In [50]:
retail_popular

,Customer ID,StockCode,Invoice,InvoiceDate,Quantity,popular,products,transactions,dates,StockLabel,train_yn,CustomerLabel
0,12346,15056,499763,2010-03-02 13:08:00,3,350,22,1,1,4,1,0
1,12346,20679,499763,2010-03-02 13:08:00,1,187,22,1,1,17,1,0
2,12346,20682,499763,2010-03-02 13:08:00,1,210,22,1,1,19,1,0
3,12346,20685,513774,2010-06-28 13:53:00,1,472,22,1,1,20,1,0
4,12346,21523,513774,2010-06-28 13:53:00,1,234,22,1,1,186,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
389677,18287,84879,508581,2010-05-17 11:55:00,8,612,74,1,1,855,0,1214
389679,18287,85040,508581,2010-05-17 11:55:00,24,162,74,2,2,879,0,1214
389680,18287,85040,534346,2010-11-22 11:51:00,48,162,74,2,2,879,0,1214
389682,18287,85199,508581,2010-05-17 11:55:00,24,170,74,1,1,901,0,1214


In [54]:
label_ = retail_popular.groupby(['StockLabel', 'StockCode'])['InvoiceDate'].nunique().\
            reset_index()[['StockLabel', 'StockCode']]

In [55]:
label_

,StockLabel,StockCode
0,0,10002
1,1,10135
2,2,15036
3,3,15044
4,4,15056
...,...,...
903,903,85216
904,904,85226
905,905,85227
906,906,85231


In [79]:
import csv
desc_ = retail_raw.groupby('StockCode').agg({'Description': 'unique'})
prod_desc = label_.merge(desc_, left_on = 'StockCode', right_index = True, how = 'left')
prod_desc['Description'] = prod_desc['Description'].apply(lambda x: ','.join(map(str, x)))
prod_desc.to_csv('../../data/retail_prod_desc.csv', index=False, sep= '\t', quoting=csv.QUOTE_ALL)

In [46]:
retail_popular.to_csv('../../data/retail_popular.csv', index=False)

In [26]:
retail_group = retail_popular.groupby(['CustomerLabel', 'StockLabel', 'train_yn'])['dates'].max().reset_index()
retail_group
#거래 목적, 2010.01 ~ 2011.03, 2010.01~ 12월 2011.01~03  상품별 판매수를 예측.

,CustomerLabel,StockLabel,train_yn,dates
0,0,4,0,1
1,0,4,1,1
2,0,17,0,2
3,0,17,1,1
4,0,19,1,1
...,...,...,...,...
183067,2464,791,1,1
183068,2464,801,1,1
183069,2464,805,1,1
183070,2464,842,1,1


In [27]:
retail_popular

,Customer ID,StockCode,Invoice,InvoiceDate,Quantity,popular,products,transactions,dates,StockLabel,train_yn,CustomerLabel
0,12346,15056,499763,2010-03-02 13:08:00,3,350,22,1,1,4,1,0
1,12346,20679,499763,2010-03-02 13:08:00,1,187,22,1,1,17,1,0
2,12346,20682,499763,2010-03-02 13:08:00,1,210,22,1,1,19,1,0
3,12346,20685,513774,2010-06-28 13:53:00,1,472,22,1,1,20,1,0
4,12346,21523,513774,2010-06-28 13:53:00,1,234,22,1,1,186,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
389677,18287,84879,508581,2010-05-17 11:55:00,8,612,74,1,1,855,0,1214
389679,18287,85040,508581,2010-05-17 11:55:00,24,162,74,2,2,879,0,1214
389680,18287,85040,534346,2010-11-22 11:51:00,48,162,74,2,2,879,0,1214
389682,18287,85199,508581,2010-05-17 11:55:00,24,170,74,1,1,901,0,1214
